In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [ ]:
import pandas as pd
import numpy as np
import os
import re
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import math

In [ ]:
df_raw = pd.read_csv('/content/drive/My Drive/Colab Notebooks/hw2/train.csv')


In [ ]:
df_raw["SalePrice"] = np.log(df_raw.SalePrice)


In [ ]:
import re
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [ ]:
df_compare = df_raw.copy()



In [ ]:
def add_datepart(df, dt_name, drop=True, time=False):
    "Creates new columns from our datetime column"
    
    
    dt_column = df[dt_name]
    column_dtype = dt_column.dtype
    

    targ_name = re.sub('[Dd]ate$', '', dt_name)
    
    # attributes are normally in lower case but we wrote this way because we will use it in columns' name too
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    
    if time: 
        attr = attr + ['Hour', 'Minute', 'Second']
        
    
    #Sorry curse of dimensionality, maybe another time
    for a in attr: 
        df[targ_name + a] = getattr(dt_column.dt, a.lower())
        
    # how much time passed, we will divide by 10^9 because it is in the nanosecond format
    df[targ_name + 'Elapsed'] = dt_column.astype(np.int64) // 10 ** 9
    
    if drop: 
        df.drop(dt_name, axis=1, inplace=True)

In [ ]:
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c):
            df[n] = c.astype("category").cat.as_ordered()

In [ ]:
def apply_cats(df, train):
    for n, c in df.items():
        if train[n].dtype == "category":
            df[n] = pd.Categorical(c, categories = train[n].cat.categories, ordered = True)

In [ ]:
df = df_raw.copy()

In [ ]:
train_cats(df)
train_cats(df_compare)

In [ ]:
df

In [ ]:
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum:
            df[name+"_na"] = pd.isnull(col)
        df[name] = col.fillna(col.median())

In [ ]:
def numericalize(df, col, name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes+1

In [ ]:
def proc_df(df, y_fld):
    
    y = df[y_fld].values
    df.drop([y_fld], axis = 1, inplace = True)
    
    for n, c in df.items():
        fix_missing(df, c, n)
        
    for n, c in df.items():
        numericalize(df, c, n)
    
    res = [df, y]
    

    
    return res

In [ ]:
df, y = proc_df(df, 'SalePrice')

In [ ]:
df_compare, y_compare = proc_df(df_compare, 'SalePrice')

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

In [ ]:
def split_train_val(df,n): 
    
    return df[:n].copy(), df[n:].copy()

In [ ]:
n_valid = 12000  
n_train = len(df)-n_valid
X_train, X_valid = split_train_val(df_compare, n_train)
y_train, y_valid = split_train_val(y_compare, n_train)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
def rmse(x,y): 
    return math.sqrt(((x-y)**2).mean())